# Introdução

O campo da física de altas energias busca compreender as quatro principais forças que regem as interações entre partículas. As informações necessárias para este estudo são coletadas a partir do estudo de uma classificação específica de partículas, que são produzidas em condições extremas.

A única forma conhecida de se gerar e analisar tais partículas é por meio de colisões realizadas em aceleradores de partículas, porém nem toda colisão gera dados relevantes. Na realidade, o Grande Colisor de Hadrons (LHC) gera em média três partículas de interesse (denominadas partículas exóticas) a cada $10^9$ colisões. As colisões restantes geram partículas denominadas background, e não são utilizadas.

Dada a quantidade de colisões realizada por hora, e a complexidade de se analisar os dados obtidos, há um grande interesse em desenvolver uma forma automatizada de se diferenciar partículas exóticas das background. Para realizar esta diferenciação, são utilizados 18 dados. Destes, 8 são dados cinéticos coletados por detectores dentro do próprio acelerador de partículas, enquanto os 10 restantes são calculados em função dos anteriores.

O dataset analizado, denominado SUSY, apresenta um total de cinco milhões de instâncias de partículas teóricas. Cada uma possui 18 variáveis numéricas representando os dados utilizados para a diferenciação entre partículas exóticas e background, e uma variável booleana que representa a **qual destas classificações** a partícula em questão se encaixa. Os dados do dataset SUSY foram gerados a partir de simulações Monte Carlo, ou seja, não são produto de colisões reais.

# Objetivos
## Geral
Leonardo
    Determinar modelo de predição para classificar se a particula é exotica ou não.
## Especificos
Leonardo
    Determinar a melhor metrica para avaliar os resultados exemplos (precisão, acurácia, falso positivo, falso negativo) e porque

# Informações do Dataset
info, head, tail, describe
Akiyoshi

# Algoritmos de Classificação

KNN,SGD,KA,GBC colocar imagem do sklearnin de seleção dos algoritmos  
Rafael, Adrianne

# Análise das Features
Gesiel falar sobre as variações dos modelos e pré concluir
## Pré Processamento
Gesiel
## Redução de Features
Afonso, João Pedro
## Teste Redução
    Testar as features com os algoritmos SGD, GBC de classificação
    Afonso
## Conclusão Análise Features
    Afonso
Concluir que é melhor não reduzir as features

# conclusão
## Algoritmo utilizado
Thiago
## Resultados Obtidos
Thiago
    Apresentar resultados obtidos, gráficos e resultados da mensuração